# OSNAP Parcels Experiments


## Technical preamble

In [ ]:
%matplotlib notebook

In [ ]:
# import matplotlib.colors as colors
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime, timedelta
# import seaborn as sns
# from matplotlib.colors import ListedColormap
import cmocean as co
import pandas as pd
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy



In [ ]:
## Parameters
# Project path
project_path = Path.cwd() / '..' / '..' 
project_path = project_path.resolve()

# model mask file
data_path = Path("data/external/iAtlantic/")
experiment_name = "VIKING20X.L46-KKG36107B"
mesh_mask_file = project_path / data_path / "mask" / experiment_name / "1_mesh_mask.nc"


## plotting functions

In [ ]:
# range of stations from west to east, stations 0-12. Python indexing.

def plotDepthContoursCartopy():
    
    central_lon, central_lat = -30, 55
    fig, ax = plt.subplots(figsize = (10,8),subplot_kw={'projection': ccrs.Orthographic(central_lon, central_lat)})
    extent = [-60, 0, 40, 70]
    ax.set_extent(extent)
    ax.gridlines()
    ax.coastlines(resolution='50m')


    cs = depth.isel(y=slice(1750, 2499), x=slice(0, 2404)).plot.contour(ax=ax,transform=ccrs.PlateCarree(),
        x="nav_lon", y="nav_lat", colors = 'grey', levels = [1000],zorder=1
    );
#     depth.isel(y=slice(1400, 2499), x=slice(0, 2404)).plot.contour(ax=ax,transform=ccrs.PlateCarree(),
#         x="nav_lon", y="nav_lat", colors = 'k', levels = [1],zorder=3
#     );

    return cs

In [ ]:
# range of stations from west to east, stations 0-12. Python indexing.

def plotDepthContoursGrid():
    
    fig, ax = plt.subplots(figsize = (10,6))

    cs = depth.isel(y=slice(1500, 2499), x=slice(500, 2200)).plot.contour(ax=ax,
         colors = 'grey', levels = [1000],zorder=1
    );

    cs1 = depth.isel(y=slice(1500, 2499), x=slice(500, 2200)).plot.contour(ax=ax,
         colors = 'k', levels = [0.5],zorder=1
    );


# need to locate the right contour (use trial and error). each separate contour has separate index
    for i in range(1):
        v = cs.collections[0].get_paths()[0].vertices
        x = v[:,0]
        y = v[:,1]

        ax.plot(x,y,color='r')
    return cs

## Load data

In [ ]:
mesh_mask = xr.open_dataset(mesh_mask_file)
mesh_mask = mesh_mask.squeeze()
mesh_mask = mesh_mask.set_coords(["nav_lon", "nav_lat", "nav_lev"])

bathy = mesh_mask.mbathy.rename("number of water filled points")

depth = (mesh_mask.e3t_0 * mesh_mask.tmask).sum("z")
# display(mesh_mask)

In [ ]:
display(mesh_mask)

In [ ]:
cs = plotDepthContoursGrid()

In [ ]:
vs = cs.collections[0].get_paths()[0].vertices
fig, ax = plt.subplots(figsize = (10,5))
cs1 = depth.isel(y=slice(1500, 2499), x=slice(500, 2200)).plot.contour(ax=ax,
     colors = 'k', levels = [0.5],zorder=1
);

ax.plot(vs[:,0],vs[:,1],marker='+')

ax.grid()

fs = []
v = vs[0]
if np.abs(np.round(v[0])-v[0]) < 0.0000001:
    a = [np.round(v[0])-1,np.trunc(v[1])]
    b = [np.round(v[0]),np.trunc(v[1])]
    fs.append(b)
    fs.append(a)

else:
    a = [np.trunc(v[0]),np.round(v[1])-1]
    b = [np.trunc(v[0]),np.round(v[1])]
    fs.append(a)
    fs.append(b)



for v in vs[1:]:
    if np.abs(np.round(v[0])-v[0]) < 0.0000001:
        a = [np.round(v[0])-1,np.trunc(v[1])]
        b = [np.round(v[0]),np.trunc(v[1])]
        if a == fs[-1]:
            fs.append(b)
        else:
            fs.append(a)
        
    else:
        a = [np.trunc(v[0]),np.round(v[1])-1]
        b = [np.trunc(v[0]),np.round(v[1])]
        if a == fs[-1]:
            fs.append(b)
        else:
            fs.append(a)
        
# for plotting offset to f points (f(i,j) is NE of T(i,j))
fs = np.array(fs)+0.5

# remove duplicates np.unique sorts by x value, want to retain original order, hence faff

# fsu,indices = np.unique(np.array(fs),axis=0,return_index=True)
# fsn=fs[np.sort(indices)]

ax.plot(fs[:,0],fs[:,1],color='C1')


In [ ]:
display(mesh_mask)
